## Data transforming for applying EEGlearn on them
Input: several files .mat with 3D matrix (time x channels x trials) 
Output: one file .mat with 2D matrix (trials x features)

Here is a preparation of data to use EEGlearn
https://github.com/pbashivan/EEGLearn
as discribed in this paper:
https://arxiv.org/abs/1511.06448

In [1]:
from __future__ import print_function
import scipy.io
import scipy.fftpack as sfp
import numpy as np
import os

In [34]:
T = 0.5 # Period of one time window (seconds)
elNum = 19 # Number of electrodes
win = 3 # Number of time windows
data = []
subNums = []
trial_subNums = []
dir = 'C:/Users/alena/Desktop/homed/laba/data'
features = np.array([]).reshape(0, elNum * 3 * win + 1) # Array for features, (3 frequency bands*windows) + 1 for label

In [35]:
for subdir in os.listdir(dir):
    for file in os.listdir(dir + '/' + subdir):
        subNums.append(int(subdir))
        data.append(scipy.io.loadmat(dir + '/' + subdir + '/' + file))
        print(dir + '/' + subdir + '/' + file + ' is loaded')

C:/Users/alena/Desktop/homed/laba/data/25/eegNT.mat is loaded
C:/Users/alena/Desktop/homed/laba/data/25/eegT.mat is loaded
C:/Users/alena/Desktop/homed/laba/data/26/eegNT.mat is loaded
C:/Users/alena/Desktop/homed/laba/data/26/eegT.mat is loaded
C:/Users/alena/Desktop/homed/laba/data/27/eegNT.mat is loaded
C:/Users/alena/Desktop/homed/laba/data/27/eegT.mat is loaded
C:/Users/alena/Desktop/homed/laba/data/28/eegNT.mat is loaded
C:/Users/alena/Desktop/homed/laba/data/28/eegT.mat is loaded
C:/Users/alena/Desktop/homed/laba/data/29/eegNT.mat is loaded
C:/Users/alena/Desktop/homed/laba/data/29/eegT.mat is loaded
C:/Users/alena/Desktop/homed/laba/data/30/eegNT.mat is loaded
C:/Users/alena/Desktop/homed/laba/data/30/eegT.mat is loaded
C:/Users/alena/Desktop/homed/laba/data/32/eegNT.mat is loaded
C:/Users/alena/Desktop/homed/laba/data/32/eegT.mat is loaded
C:/Users/alena/Desktop/homed/laba/data/33/eegNT.mat is loaded
C:/Users/alena/Desktop/homed/laba/data/33/eegT.mat is loaded
C:/Users/alena/D

In [ ]:
print('Starting making 2D array from data...')
for i in range(len(data)):
    dic = data[i]
    label = 1 if 'eegT' in dic.keys() else 0
    matrix = dic.get('eegT') if label == 1 else dic.get('eegNT')   
    N = matrix.shape[0]//win # Number of points for one time window
    if elNum != matrix.shape[1]:
        raise elNumException('Wrong number of electrodes!')
    line = np.zeros((1,elNum*3*win + 1), dtype = 'float64') # (3 frequency bands (theta, alpha, beta)*windows) plus 1 
                                                            # for label
    
    for _ in range(matrix.shape[2]):
        trial_subNums.append(subNums[i]) # subject numbers associated with each trial (for leave-subject-out
                                         #  cross validation).
        
    print('!',end='')
    for trialNum in range(matrix.shape[2]):
        for el in range(elNum):
            trial = matrix[:, el, trialNum]
            for w in range(win):
                timewin = trial[w * N : (w+1) * N]
                # trial -= np.mean(trial) to get rid of zero frequency in fft (e.g. for visualization of fft)
                A = abs(sfp.fft(timewin))/N # Real amplitudes of sinusoids
                Hz = np.array([k/T for k in range(N)]) # Frequences in hertz
                # Computing the sum of squared absolute values within each of the three
                # frequency bands of theta (4-7Hz), alpha (8-13Hz), and beta(13-30Hz)
                theta = ((A[[np.all([Hz[k] < 7, Hz[k] >= 4]) for k in range(N)]])**2).sum()
                alpha = ((A[[np.all([Hz[k] < 13, Hz[k] >= 7]) for k in range(N)]])**2).sum()
                beta = ((A[[np.all([Hz[k] <= 30, Hz[k] >= 13]) for k in range(N)]])**2).sum()
                line[:,elNum*3*w + el], line[:,elNum*(3*w + 1)+ el], line[:,elNum*(3*w + 2)+ el] = theta, alpha, beta
        line[:,-1] = label
        features = np.concatenate((features, line))
        print('.',end='')
    print()
print('Array is made!')

Starting making 2D array from data...
!.............................................................................................
!............................................................................................................................................................................................................................................
!............................................................................................
!...................................................................................................................................................................................................................................................................................................................................
!..................................................................................................................................................
!.............................................................

In [37]:
print('Saving FeatureMat_timeWin.mat...')
featureMatrix = np.array(features, ndmin =2)
featureDict = dict([('features', featureMatrix), ])
scipy.io.savemat('C:/Users/alena/Desktop/homed/laba/normData/FeatureMat_timeWin.mat', featureDict)

print('Saving trial_subNums.mat...')
trial_subMatrix = np.array(trial_subNums, ndmin =2)
trial_subDict = dict([('subjectNum', trial_subMatrix), ])
scipy.io.savemat('C:/Users/alena/Desktop/homed/laba/normData/trials_subNums.mat', trial_subDict)

Saving FeatureMat_timeWin.mat...
Saving trial_subNums.mat...
